PETERS Gauthier (pega1012) / FIMBEL Nicolas (fini1024)


Da Dateien nicht mehr als 256MB in Ilias nicht erlaubt wurden, konnten wir nicht die Bilder mit der Hausarbeit geben. Das Tensorflow Verfahren wird nicht richtig laufen, sowie Merkmale_OpenCV. Die selbst programmierte Verfahren können verwenden werden, da sie die CSV Dateien benutzen, die schon ergestellt wurden. 


# Merkmale aus Bilder herausziehen und Export als CSV Dokument

Mit diesem Programm wollen wir verschiedene Merkmale von den Bilder von Kämme und Bürste erleitern und dann wollen wir eine CSV Datei mit diesen Merkmalen erstellen.

## I) Import von Bibliotheken und von den Bilder

In [1]:
# Import von Bibliotheken
import cv2;
import numpy as np;
import pathlib;
import csv;

## II) Pfad der Bilder

Anhand der Bibliothek **pathlib** können wir den Pfad der Bilder einfach definieren. Darunter ist der Pfad "data_dir" der Pfad des Ordners, der die 2 unter Ordner **Kamm** und **Buerste** enthält.

Dan können wir die gesamte Anzahl der Bilder geben.

In [2]:
# Pfad der Bilder 
data_dir = pathlib.Path(r"Bilder/Verfahrenbilder")
print("Pfad der Bilder : ", data_dir)

# Anzahl von Bilder
image_count = len(list(data_dir.glob('*/*.jpg')))
print('Anzahl von Bilder :', image_count)

Pfad der Bilder :  Bilder\Verfahrenbilder
Anzahl von Bilder : 1306


## III) Erste Bilder zeigen

Um die Behandlung der Bilder zu vereinfachen, wir haben die Folgende Etappe gefolgt :
- Alle .zip Datei in Ilias herunterladen.
- Alle Bilder in das JPG-Format konvertieren und umbennen(mit Hilfe des XnView Softwares).

Nach diesen Etappen erhalten wir einen Order **Bilder**, der 2 Unterordner enthält : **Buerste** und **Kamm**. 

Jetzt benutzen wir die Bibliothek **openCV-Python**, um die erste Bilder zu zeigen.

In [3]:
# Erstes Bild von Bürste zeigen
Buerste = list(data_dir.glob('Buerste/*'))  # Ordner der Bilder von Bürste
print(str(Buerste[0]))
img = cv2.imread(str(Buerste[0]))           # Bild mit OpenCV öfnen
cv2.imshow("erstes Bild von Buerste", img)  # Bild zeigen
cv2.waitKey(0);
cv2.destroyAllWindows()

Bilder\Verfahrenbilder\Buerste\Buerste_0001.jpg


In [4]:
# Erstes Bild von Kamm zeigen
Kamm = list(data_dir.glob('Kamm/*'))        # Ordner der Bilder von Kämme
print(str(Kamm[3]))
img = cv2.imread(str(Kamm[3]))              # Bild mit OpenCV öfnen
cv2.imshow("erstes Bild von Kamm", img)     # Bild zeigen
cv2.waitKey(0);
cv2.destroyAllWindows()

Bilder\Verfahrenbilder\Kamm\Kamm_0004.jpg


## IV) Bilder skalieren

Die Bilder haben verschiedene Herkünfte (Internet, verschiedene Fotoapparate). Deswegen hat jedes Bild ihres eigenes Ausmaß. Wir benutzen also OpenCV, um alle Bilder zu skalieren. 

Wir wählen ein neues konstantes Ausmaß, dass das selbe für jedes Bild wird.


In [5]:
# Neue Ausmaß der Bilder
width = 500            # Breite in Pixel
height = 500            # Höhe in Pixel
dim = (width, height)   # neues Ausmaß der Bilder

In [7]:
#### TEST BILD ####
img = cv2.imread(str(Buerste[347]))
###################

imgResized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
cv2.imshow("Risized", imgResized)     # Bild zeigen
cv2.waitKey(0);
cv2.destroyAllWindows()

## V) Merkmale von Bilder identifizieren 

Zuerst werden wir einen Wert erläutern, der sagt, ob das Bild ein Kamm oder eine Bürste ist. Dieser Wert wird in der letzten Spalte der CSV Datei geschrieben:

**- 0 wenn das Bild einen Kamm ist**

**- 1 wenn das Bild eine Bürste ist**

Dann müssen wir die Bilder erarbeiten, um Merkmale zu identifizieren. Um die Merkmale zu erläutern werden wir eine Probe von Bilder benutzen.

### 1. Normierung der Bilder

Erstens normieren wir die Bilder, um das Rauschen zu reduzieren und den Kontrast zu verbessern. Tatsächlich reduzieren wir mit der Normierung das Hoch- und Tiefrauschen von der Bilder.


In [8]:
imgNormiert = cv2.normalize(imgResized,None,0,255,cv2.NORM_MINMAX) 
stack = np.hstack((imgResized, imgNormiert ))       # Beider Bilder in der selben Fenster zeigen

cv2.imshow("origineles und normiertes Bilder", stack)     
cv2.waitKey(0);
cv2.destroyAllWindows()

### 2. Bilder in Grau umwandeln

Wir wollen dannach die Kontouren der Bilder suchen. Dazu sollen wir zuerst die Bilder in Grau umwandeln, d.h., dass die Bilder nur mir einer Kanal (statt 3 mit RGB) beschreiben werden.

In [9]:
# Bild in Grau umwandeln und zeigen
imgGrau = cv2.cvtColor(imgNormiert,cv2.COLOR_BGR2GRAY)

cv2.imshow("imgGrau", imgGrau)     
cv2.waitKey(0);
cv2.destroyAllWindows()

### 3. Bilder bilaterale filtern

Um die scharfe Kante zu behalten aber das Bilder mehr zu egalisieren benutzen wir das bilaterales Filter:

In [10]:
# Bild filtern und zeigen
imgBlur = cv2.bilateralFilter(imgGrau,9,100,100)

cv2.imshow("imgGrau und imgBlur", np.hstack((imgGrau, imgBlur)))     
cv2.waitKey(0);
cv2.destroyAllWindows()

### 4. Kante Erkennung : Edge detection

Wir haben jetzt graue Bilder, die gefiltert wurden. Wir können alsro eine Kante Erkennung durchführen, um alle Kante in jedem Bild zu erkennen. 

Wir bilden eine Funktion, die den Midianwert des Bildes zuerst berechnent, um dann die Grenze in der *cv2.canny* Funktion automatisch zu bestimmen. 

In einigen Bilder ist der Farbeunterschied zwischen dem Hinter- und Verdergrund gering. Dazu kann die *cannyEdge* Funktion in diesen Fälle keine Kontur finden. Damit erhielten wir eine Fehlermeldung, wenn wir die größte Konturen suchten. 

Deshalb haben wir die Funktion erweitert, so dass wir auch diese Fälle betrachten könnten. Tatsächlich soll die obere und untere Grenze (upper und lower) kleiner sein. Wir haben also eine Grenze definiert, so dass wir immer Konture erhalten. 

In [11]:
def cannyEdge(img):
    sigma = 0.8         # Parameter, um die untere und obere Grenze zu variieren
    m = np.median(img)  # Medianwert
    lower = int(max(0, (1.0 - sigma) * m))   # lower threshold (Grenze)
    upper = int(min(255, (1.0 + sigma) * m)) # upper threshold (Grenze)
    imgCanny = cv2.Canny(img, lower, upper,L2gradient=True)  # OpenCV, die de Kante sucht
    contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_TC89_L1)
    while len(contours)<20:    # Schleife, um eine miniale Anzahl von Konturen zu erhalten
        lower -= 10
        upper -= 10
        imgCanny = cv2.Canny(img, lower, upper,L2gradient=True)
        contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_TC89_L1)
    return imgCanny,contours

In [12]:
# Kante Erkennung
imgCanny = cannyEdge(imgBlur)[0]

cv2.imshow("Kanny", imgCanny)
cv2.waitKey(0);
cv2.destroyAllWindows()

Wir können mit einer Funktion von OpenCV die Kanten auf dem Bild zeichnen.

In [13]:
contours = cannyEdge(imgBlur)[1]
cv2.drawContours(imgResized,contours,-1,(255,0,0),2)   # Alle Konture in Blau zeichen

cv2.imshow("Kante", imgResized) 
cv2.waitKey(0);
cv2.destroyAllWindows()

### 5. Umfang der größere Kontur

Wir wollen in allen Konturen die größte Kontur herausziehen, un dann ihren Flächeninhalt berechnen. Dazu benutzen wir die Funktion *cv2.contourArea* von OpenCV.

In [14]:
# Umfang von der größten Contour berechnen
c_max = max(contours, key = cv2.contourArea)
c_maxArea = cv2.contourArea(c_max)            # Flächeninhalt der größte Kontur 
c_maxLength = cv2.arcLength(c_max,True)       # Länge der größten Kontur
cv2.drawContours(imgResized,[c_max],-1,(0,120,255),2)   # größte Kontur in Orange
print('Flächeninhalt größter Kontur = ', c_maxArea)
print('Länge größter Kontur = ', c_maxLength)
cv2.imshow("Kante", imgResized) 
cv2.waitKey(0);
cv2.destroyAllWindows()

Flächeninhalt größter Kontur =  27812.0
Länge größter Kontur =  834.482091665268


### 6. Convexity Hull 

Wir wollen jetzt die sogenannte Konvexe Hülle (Convex Hull) berechnen und bzw. den Flächeninhalt dieser besonderen Kontur.


In [15]:
# hull Kontur berechnen
hull = cv2.convexHull(c_max)
hullArea = cv2.contourArea(hull)       # Flächeninhalt der hull Kontur
imgHull = cv2.drawContours(imgResized, [hull], -1, color=(0, 0, 255), thickness=5)  

print('Hull Area = ', hullArea)
cv2.imshow("Hull", imgResized) 
cv2.waitKey(0);
cv2.destroyAllWindows()

Hull Area =  33362.5


### 7. Solidity

Wir können die sogenannte solidity mit der Formel $$solidity = \displaystyle\frac{c_{maxArea}}{hullArea}$$ definieren. Es ist der Quotient von dem Flächeninhalt der größten Kontur mit dem Flächeninhalt des Convexity Hulls.

In [16]:
solidity = c_maxArea/hullArea
print('solidity = ',solidity)

solidity =  0.8336305732484076


### 8. Circularity

Die Zirkularität ist mit der Formel $$circularity = \displaystyle\frac{4\pi *c_{maxArea}}{c_{maxLength^2}}$$ definiert. Es ist der Quotient von dem Flächeninhalt der größten Kontur mit dem Länge der größten Kontur.

In [69]:
# 8. Circularity
circularity= 4*np.pi*c_maxArea/(c_maxLength**2)
print('circularity = ',circularity)

circularity =  0.5018894224091988


### 9. Bounding box und Extent

Wir können für jede Kontur ein bounding box definieren. Wir interessieren uns nur für die größte contour. Deshalb berechnen wir uns das Bounding Box nur für diese Kontur. Dazu ist das sogenannte **Extent** mit der Formel $$extent=\displaystyle\frac{ c_{maxArea}}{boundingBoxArea} $$ definiert.

In [70]:
# 9. bounding box und Extent
x,y,length,heigth = cv2.boundingRect(c_max)
cv2.rectangle(imgResized,(x,y),(x+length,y+heigth),(255,255,0),2)   # Bounding box in blue zeichen
  
boundingBoxArea = length*heigth
extent = c_maxArea /  boundingBoxArea

print('Area of bounding box = ',boundingBoxArea)
print('extent = ',extent)

cv2.imshow("Hull", imgResized) 
cv2.waitKey(0);
cv2.destroyAllWindows()

Area of bounding box =  44697
extent =  0.6222341544175224


### 10. Keypoints des Objektes finden

Wir benutzen jetzt einen Algorithmus, der die sogenannte "Keypoints" in den Bilder finden soll. Dieser Algorithmus ist OBR benannt und wurde 2011 gegründet. [https://docs.opencv.org/master/d1/d89/tutorial_py_orb.html]

In der CSV Datei werden wir die Anzahl von Keypoints schreiben. 

In [71]:
orb = cv2.ORB_create()
keyPoints, description = orb.detectAndCompute(imgCanny, None)
imgKeypoints = cv2.drawKeypoints(imgCanny, keyPoints, img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
print('Keypoints:', len(keyPoints))

cv2.imshow("Keypoints", imgKeypoints) 
cv2.waitKey(0);
cv2.destroyAllWindows()

Keypoints: 419


### 11. Polygonale Schätzung

Jetzt benutzen wir die Funktion *cv2.approxPolyDP*. Diese Funktion annährt eine Kontur mit einem Polynom. Wir werden hier auch die Anzahl an Punkte in der CSV Datei schreiben.  

In [72]:
c_maxLength = cv2.arcLength(c_max,True)                           # Länge der größten Kontur
approx = cv2.approxPolyDP(c_max,0.005*c_maxLength,True)           # Koordinaten der Punkte der Schätzung
print(len(approx))                                                # Anzahl von Punkten, die mit der Schätzung gefunden wurden
cv2.polylines(imgResized, approx, True, (0,255,0), 6,lineType=8)  # Zeichen der Punkte in Grün

cv2.imshow("Approx", imgResized) 
cv2.waitKey(0);
cv2.destroyAllWindows()

16


## VI) CSV Datei schreiben

### 1) Funktion für Merkmale
Um eine CSV Datei zu schreiben programmieren wir zuerst eine Funktion, die die Merkmale einer Bilder berechnent und in einer Liste schreibt.

In [73]:
def merkmale(imgPath,klasse):      # Argument : Pfad des Bildes und Klasse (0:Kamm oder 1:Bürste)
    imgOrigin = cv2.imread(str(imgPath))
    # Skalierung des Bildes
    width = 500            # Breite in Pixel
    height = 500            # Höhe in Pixel
    dim = (width, height)   # neues Ausmaß der Bilder
    
    imgResized = cv2.resize(imgOrigin, dim, interpolation = cv2.INTER_AREA)
    # 1. Normierung des Bildes
    imgNormiert = cv2.normalize(imgResized,None,0,255,cv2.NORM_MINMAX) 
    
    # 2. Grau Bild
    imgGrau = cv2.cvtColor(imgNormiert,cv2.COLOR_BGR2GRAY)
    
    # 3. Bild filtern
    imgBlur = cv2.bilateralFilter(imgGrau,9,80,80)
    
    # 4. Canny Bild und Konturen 
    imgCanny, contours = cannyEdge(imgBlur)
        
    # 5. Größte Kontur
    c_max = max(contours, key = cv2.contourArea)
    c_maxLength = cv2.arcLength(c_max,True)       # Länge der größten Kontur
    c_maxArea = cv2.contourArea(c_max)
    
    # 6. Hull
    hull = cv2.convexHull(c_max)
    hullArea = cv2.contourArea(hull)
    
    # 7. Solidity
    solidity = c_maxArea/hullArea
    
    # 8. Circularity
    circularity= 4*c_maxArea/(c_maxLength**2)

    # 9. bounding box und Extent
    x,y,length,heigth = cv2.boundingRect(c_max)
    boundingBoxArea = length*heigth
    extent = c_maxArea /  boundingBoxArea
                            
    # 10. Keypoints des Objektes finden
    orb = cv2.ORB_create()
    keyPoints, description = orb.detectAndCompute(imgCanny, None)
    keyPointsAnzahl = len(keyPoints)
    
    # 11. Polygonale Schätzung
    approx = cv2.approxPolyDP(c_max,0.005*c_maxLength,True)
    approxAnzahl = len(approx)           # Anzahl an Punkte in der polynomalen Schätzung
    
    # Merkmale in der Liste schreiben 
    merkmale = [c_maxLength,c_maxArea,circularity,solidity,extent,keyPointsAnzahl,approxAnzahl,klasse]
    return merkmale

In [74]:
# Prüfung der Funktion mit einem Bild
print(merkmale(Buerste[3],0))

[2251.7386873960495, 306.0, 0.00024140454292608128, 0.006599521211206246, 0.005047422680412371, 500, 31, 0]


### 2. CSV Datei schreiben 

#### Bilder für das Verfahren

Wir verwenden wir die Funktion *merkmale* auf jedes Bild von Kamm und Bürste, um die CSV Datei zu bilden.

In [26]:
with open("merkmale.csv", 'w', newline='') as f:
    writer = csv.writer(f)                    # Erstellung des Schreibers
    for path in Kamm:                         # Schleife über alle Bilder von Bürste
        writer.writerow(merkmale(path,0))     # Neue Zeile mit den Merkmale addieren
    for path in Buerste:                      # Schleife über alle Bilder von Bürste
        writer.writerow(merkmale(path,1))     # Neue Zeile mit den Merkmale addiere
    print('done')

done


#### Testbilder

Wir wollen die verschiedene Verfahren mit Testbilder, d.h. Bilder, die extern von dem Verfahren sind. Dazu haben wir 3 Bilder von Kämme und 3 Bilder von Bürste in den Ordnern **Buerste_test** bzw. **Kamm_test** gestellt. Zuerst definieren die Pfade dieser Ordnern:

In [41]:
# Pfad der Testbilder
testData_dir = pathlib.Path(r"Bilder/Testbilder")
print("Pfad der Bilder : ", testData_dir)

# Anzahl von Bilder
test_image_count = len(list(testData_dir.glob('*/*.jpg')))
print('Anzahl von Testbilder :', test_image_count)

Buerste_test = list(testData_dir.glob('Buerste_test/*'))  # Ordner der Bilder von Bürste
Kamm_test = list(testData_dir.glob('Kamm_test/*'))     # Ordner der Bilder von Bürste

Pfad der Bilder :  Bilder\Testbilder
Anzahl von Testbilder : 12


In [45]:
# CSV Datei für die Testbilder
with open("merkmale_test.csv", 'w', newline='') as f:
    writer = csv.writer(f)                    # create the csv writer
    for path in Kamm_test:                      # Schleife über alle Bilder von Bürste
        writer.writerow(merkmale(path,0))
    for path in Buerste_test:                      # Schleife über alle Bilder von Bürste
        writer.writerow(merkmale(path,1))
    print('done')

done
